In [3]:
import gym
import random
import numpy as np
from collections import deque
from keras.models import Sequential
from keras.layers import Dense
from keras.optimizers import Adam

2023-04-18 08:22:04.203076: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [8]:
class DQNAgent:
    def __init__(self, state_size, action_size):
        self.state_size = state_size
        self.action_size = action_size
        self.memory = deque(maxlen=20000)
        self.gamma = 0.99
        self.epsilon = 1.0
        self.epsilon_min = 0.2
        self.epsilon_decay = 0.999
        self.learning_rate = 0.0001
        self.model = self._build_model()

    def _build_model(self):
        model = Sequential()
        model.add(Dense(128, input_dim=self.state_size, activation='relu'))
        model.add(Dense(128, activation='relu'))
        model.add(Dense(self.action_size, activation='linear'))
        model.compile(loss='mse', optimizer=Adam(lr=self.learning_rate))
        return model

    def remember(self, state, action, reward, next_state, done):
        self.memory.append((state, action, reward, next_state, done))

    def act(self, state):
        if np.random.rand() <= self.epsilon:
            return random.randrange(self.action_size)
        act_values = self.model.predict(state)
        return np.argmax(act_values[0])

    def replay(self, batch_size):
        minibatch = random.sample(self.memory, batch_size)
        for state, action, reward, next_state, done in minibatch:
            target = reward
            if not done:
                target = (reward + self.gamma * np.amax(self.model.predict(next_state)[0]))
            target_f = self.model.predict(state)
            target_f[0][action] = target
            self.model.fit(state, target_f, epochs=1, verbose=0)
        if self.epsilon > self.epsilon_min:
            self.epsilon *= self.epsilon_decay

    def load(self, name):
        self.model.load_weights(name)

    def save(self, name):
        self.model.save_weights(name)


In [9]:
env = gym.make('CartPole-v1')
state_size = env.observation_space.shape[0]
action_size = env.action_space.n
agent = DQNAgent(state_size, action_size)
print('state size', state_size)
print('action size', action_size)

done = False
batch_size = 32
EPISODES = 1000

state size 4
action size 2


In [ ]:
for e in range(EPISODES):
    state = env.reset()
    print(state)
    state = np.reshape(state, [1, state_size])
    print('new state', state)
    for time in range(500):
        # env.render()
        action = agent.act([state])
        next_state, reward, done, _ = env.step(action)
        reward = reward if not done else -10
        next_state = np.reshape(next_state, [1, state_size])
        agent.remember(state, action, reward, next_state, done)
        state = next_state
        if done:
            print("episode: {}/{}, score: {}, e: {:.2}".format(e, EPISODES, time, agent.epsilon))
            break
    if len(agent.memory) > batch_size:
        agent.replay(batch_size)
    if e % 100 == 0:
        agent.save("lunar_lander-dqn.h5") # lol wrong name here

[ 0.01646276 -0.03200918  0.04143762  0.01632202]
new state [[ 0.01646276 -0.03200918  0.04143762  0.01632202]]
episode: 0/1000, score: 24, e: 1.0
[ 0.00860815 -0.01877899 -0.02964288  0.01685256]
new state [[ 0.00860815 -0.01877899 -0.02964288  0.01685256]]
episode: 1/1000, score: 14, e: 1.0
1/1 [==============================] - 0s 31ms/step
[-0.00264927 -0.00089259  0.00946645  0.00933072]
new state [[-0.00264927 -0.00089259  0.00946645  0.00933072]]
episode: 2/1000, score: 30, e: 1.0
1/1 [==============================] - 0s 37ms/step
[ 0.04659918  0.03501585 -0.016171    0.04161484]
new state [[ 0.04659918  0.03501585 -0.016171    0.04161484]]
episode: 3/1000, score: 13, e: 1.0
1/1 [==============================] - 0s 28ms/step


1/1 [==============================] - 0s 44ms/step
[-0.00211357  0.00587764 -0.04813442 -0.02420195]
new state [[-0.00211357  0.00587764 -0.04813442 -0.02420195]]
episode: 4/1000, score: 21, e: 1.0
1/1 [==============================] - 0s 39ms/step
[ 0.01953335 -0.04364393 -0.03995696 -0.0087153 ]
new state [[ 0.01953335 -0.04364393 -0.03995696 -0.0087153 ]]
episode: 5/1000, score: 34, e: 1.0
1/1 [==============================] - 0s 54ms/step


1/1 [==============================] - 0s 32ms/step
[-0.04459907  0.04881554  0.00385285  0.01619631]
new state [[-0.04459907  0.04881554  0.00385285  0.01619631]]
episode: 6/1000, score: 9, e: 1.0
1/1 [==============================] - 0s 45ms/step
[ 0.04282948 -0.01674461 -0.02480228  0.04432823]
new state [[ 0.04282948 -0.01674461 -0.02480228  0.04432823]]
1/1 [==============================] - 0s 120ms/step
episode: 7/1000, score: 12, e: 0.99
1/1 [==============================] - 0s 28ms/step
[ 0.02778559  0.04817513 -0.00258144  0.00558985]
new state [[ 0.02778559  0.04817513 -0.00258144  0.00558985]]
episode: 8/1000, score: 24, e: 0.99
1/1 [==============================] - 0s 29ms/step


1/1 [==============================] - 0s 49ms/step
[-0.02298704  0.03677144  0.04272695 -0.00465129]
new state [[-0.02298704  0.03677144  0.04272695 -0.00465129]]
1/1 [==============================] - 0s 126ms/step
episode: 9/1000, score: 25, e: 0.99
1/1 [==============================] - 0s 46ms/step
[0.03192025 0.02329931 0.04922074 0.01713847]
new state [[0.03192025 0.02329931 0.04922074 0.01713847]]
1/1 [==============================] - 0s 38ms/step
episode: 10/1000, score: 36, e: 0.99
1/1 [==============================] - 0s 42ms/step


1/1 [==============================] - 0s 110ms/step
[ 0.02840015 -0.02754864  0.03451644  0.04858935]
new state [[ 0.02840015 -0.02754864  0.03451644  0.04858935]]
episode: 11/1000, score: 55, e: 0.99
1/1 [==============================] - 0s 38ms/step
[-0.01005673  0.0195688   0.01640136  0.04550994]
new state [[-0.01005673  0.0195688   0.01640136  0.04550994]]
1/1 [==============================] - 0s 42ms/step
episode: 12/1000, score: 43, e: 0.99
1/1 [==============================] - 0s 39ms/step


...

1/1 [==============================] - 0s 36ms/step
[ 0.04464263  0.0319691  -0.04143246  0.01314071]
new state [[ 0.04464263  0.0319691  -0.04143246  0.01314071]]
1/1 [==============================] - 0s 47ms/step
episode: 96/1000, score: 77, e: 0.91
1/1 [==============================] - 0s 35ms/step
[-0.02246374 -0.02147756 -0.00950146  0.00744935]
new state [[-0.02246374 -0.02147756 -0.00950146  0.00744935]]
1/1 [==============================] - 0s 49ms/step
episode: 97/1000, score: 18, e: 0.91
1/1 [==============================] - 0s 34ms/step


1/1 [==============================] - 0s 34ms/step
[ 0.02005563 -0.04485001  0.02107627  0.02045572]
new state [[ 0.02005563 -0.04485001  0.02107627  0.02045572]]
1/1 [==============================] - 0s 60ms/step
episode: 98/1000, score: 30, e: 0.91
1/1 [==============================] - 0s 24ms/step
[-0.01222692 -0.04686414 -0.04103479 -0.02322042]
new state [[-0.01222692 -0.04686414 -0.04103479 -0.02322042]]
1/1 [==============================] - 0s 25ms/step
episode: 99/1000, score: 17, e: 0.91
1/1 [==============================] - 0s 26ms/step


1/1 [==============================] - 0s 39ms/step
[ 0.02476844 -0.04470912 -0.02672893 -0.01620481]
new state [[ 0.02476844 -0.04470912 -0.02672893 -0.01620481]]
1/1 [==============================] - 0s 25ms/step
episode: 100/1000, score: 29, e: 0.91
1/1 [==============================] - 0s 39ms/step
[ 0.01398906 -0.04109595  0.04000726  0.04238008]
new state [[ 0.01398906 -0.04109595  0.04000726  0.04238008]]
1/1 [==============================] - 0s 41ms/step
episode: 101/1000, score: 17, e: 0.9
1/1 [==============================] - 0s 35ms/step


1/1 [==============================] - 0s 36ms/step
[-0.00054234 -0.02413111  0.043026    0.00118368]
new state [[-0.00054234 -0.02413111  0.043026    0.00118368]]
1/1 [==============================] - 0s 34ms/step
episode: 102/1000, score: 13, e: 0.9
1/1 [==============================] - 0s 29ms/step
[-0.03153818 -0.04257079 -0.00643428 -0.02216462]
new state [[-0.03153818 -0.04257079 -0.00643428 -0.02216462]]
1/1 [==============================] - 0s 28ms/step
episode: 103/1000, score: 17, e: 0.9
1/1 [==============================] - 0s 34ms/step
[-0.00097222 -0.04649778  0.01122827 -0.01873366]
new state [[-0.00097222 -0.04649778  0.01122827 -0.01873366]]
episode: 104/1000, score: 9, e: 0.9
1/1 [==============================] - 0s 26ms/step


1/1 [==============================] - 0s 28ms/step


In [ ]:
env = gym.make('CartPole-v1')
state = env.reset()
print(state)
state = np.reshape(state[0], [1, state_size])
print('new state', state)


In [ ]:
action = agent.act([state])

In [ ]:
env.step(action)